# Gradio Interfaces and HuggingFace Spaces

Huggingface [Spaces](https://huggingface.co/spaces) provide an easy-to-use way to explore and demo models. The platform is highly accessible, free to use, and allows you to share models without the need for the user to run any code.

The best part - you can insert your own model from huggingface, build your app with [gradio](https://gradio.app/docs/), and deploy in no time!

Let's use the model that we generated in the `4_1-text-classification-finetune-solns.ipynb` notebook and create a gradio space to demonstrate it!

## Install and Import Packages

In [13]:
%%capture
!pip install gradio transformers

In [18]:
# import necessary libraries
import gradio as gr
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import notebook_login

In [15]:
!git config --global credential.helper store

In [16]:
notebook_login()

## Load in Your Model

Next, we'll load in our model from huggingface. This should be in a HF repo under your name, probably formatted `your-username/model-name`.
We'll use the `Auto` classes to load in this model. The `Auto` classes in the Hugging Face transformers library are designed to automatically infer the correct model architecture or tokenizer based on the model checkpoint provided.

For example, below, AutoModelForSequenceClassification is specifically designed for sequence classification tasks, such as text classification or sentiment analysis (which is what bert-emotion was). If you've fine-tuned a model for a different type of task, like question answering or named entity recognition, you would need to use a different auto model class that corresponds to that task. For example, for question answering, you might use AutoModelForQuestionAnswering.

To ensure the right model class is used, you should use the appropriate auto model class based on the task your model was fine-tuned for. You can look at the config.json file associated with a model checkpoint to see the type of model. (You can also use this model name directly - but the `Auto` classes will give you more flexibility!)

[ See more about Auto classes [here](https://huggingface.co/docs/transformers/model_doc/auto#auto-classes). ]

In [17]:
# specify the model name
# replace 'your-username/model-name' with the name of your custom trained model
model_name = 'apetulante/bert-emotion'

# initialize the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Let's also define our labels so we know how to interpret the output from the model.

In [19]:
labels = {0: 'anger', 1: 'joy', 2: 'optimism', 3: 'sadness'}

## Define and Create the Gradio Interface

Next, we'll define a function that will do the sentiment analysis task for us. A lot of this should look very similar to how we did basic inferencing with Huggingface, because now that we've pushed our model there, we can grab it just like any other model!

In [31]:
# Define the prediction function
def predict_sentiment(text):
    # Tokenize the input tweet using the tokenizer
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,  # Add special tokens for BERT
        truncation=True,  # Truncate the input if it exceeds the maximum sequence length
        padding='longest',  # Pad the input sequences to the length of the longest sequence
        return_tensors='pt'  # Return PyTorch tensors
    )
    
    # Pass the tokenized inputs to the model
    outputs = model(**inputs)
    
    # Get the predicted class by finding the index of the highest logit score
    logits = outputs.logits.detach().numpy()
    predicted_class = np.argmax(logits, axis=1).item()
    
    # Map the predicted class index to the corresponding sentiment label using the labels dictionary
    sentiment_label = labels[predicted_class]
    
    # Return the predicted sentiment label
    return sentiment_label


In [32]:
predict_sentiment("okay,let's go!")

'sadness'

Let's define the Gradio interface with `sentiment_analysis` as the function that takes user inputs and generates outputs. The `inputs` argument specifies the input component, in this case a textbox where users can enter text. The `outputs` argument specifies the type of the output, in this case a simple text.

In [33]:
# Define the Gradio interface
iface = gr.Interface(
    fn=predict_sentiment,
    inputs="text",
    outputs="text",
    title="Sentiment Analysis",
    description="Enter a tweet and get its sentiment prediction.",
    examples=[
        ["I'm furious right now."],
        ["I have been feeling amazing lately!"],
        ["I think that everything is going to turn out okay."],
        ["Feeling really down today."],
    ]
)

# Run the Gradio interface
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

You may notice a "flag" option here. The flag functionality is a default feature in Gradio. When you launch a Gradio interface, you'll notice a "Flag" button alongside each input-output pair. Clicking this button allows you to flag examples where the model's output may not be correct or as expected.

We can view these flagged examples in the `log.csv` file that will be saved in the `flagged` folder to the left.

## Turn it into a Huggingface Space!

Simply turn this code into a app.py file, and create a huggingface space. Since the model is already hosted on huggingface, you should be up and running in no time! You can create a new space [here](https://huggingface.co/new-space).

You can turn this colab file into a .py file directly, but will need to rename the file to app.py and comment out these lines from the top of the notebook:

```
!git config --global credential.helper store

notebook_login()
```

You will also need to create a requirements file for the space to know what packages it needs. You can do this by simply creating a `requirements.txt` file in your space's repository and editing it with the necessary packages.

This will involve minimally adding the names of the packages we imported above, but may likely also require specific importing of torch, accelerate, and other packages the model requires.

Then, just add the app.py and requirements.txt files into the "Files" on your new gradio space, wait for it to build, and voila!

You can view my example space here: https://huggingface.co/spaces/apetulante/bert-emotion

## Optional Homework

We've just touched the surface of what gradio can do here, but there are a TON of other options of cool features to add or things to do with gradio. Try out a few on your own!

The code to create the gradio space is also fairly short. You can try giving the code to make this space to ChatGPT, and ask it to help you come up with idead or code additional features.

In [ ]:
#@title Add Confidence Information
#@markdown With each of these predictions, the model has some confidence
#@markdown that the given prediction is correct.
#@markdown It can be useful to display the relative prediction confidence
#@markdown for *all* classes, so we can know if the model was less sure of
#@markdown an answer

In [ ]:
#@title Predict in Batch
#@markdown Often, it's convenient to use a gradio space to allow
#@markdown users to predict on a batch of inputs.
#@markdown Imagine you have a text file with a new tweet to determine the sentiment
#@markdown of on each line. How can you edit this gradio space to accept
#@markdown and return a .txt file?

In [ ]:
#@title Try Visualizations
#@markdown With a batch prediction, there's an opportunity
#@markdown to try visualizations with the data.
#@markdown Try to show a pie or bar chart of the sentiments of a batch.